In [1]:
import os
import cv2

## 0. Video_of_one_Anchor 전처리 과정 설명

In [2]:
# 1. face detection이 2명 이상 되었을 때 drop out시키기
# -> 1-1. 영상 첫 장면을 사진 변환하기 
# -> 1-2. 첫 장면에서부터 얼굴 2개가 검출될 경우(인물 배경 뉴스)에는 영상 편집에서 아예 제외 
# 2. 처음에 나온 긴 침묵을 기준으로 영상 모두 자르기
# 3. 얼굴 2명이 나온 경우에 영상 자르기

# 1-1번에서 제외된 영상은 stt 기준으로 전처리 (주로 정치인 관련 영상들)
# ->'기자'라는 단어가 처음으로 언급되는 시점에 영상 자르기 
# (ex. OOO 기자 연결하겠습니다.)


# [Git readmemd에 올려뒀던 전처리 방법]
# face detection이 2명 이상 되었을 때 -> drop out
# 긴 침묵 (video_segment)을 기준으로 crop
# 만약 2로 잘 잘리지 않는다면, stt 사용 고려

## 1-1. 영상 첫 장면을 사진으로 변환하기

In [3]:
# 영상들 리스트로 만들기
MOV_Directory = './MOV'
MOV_List = os.listdir(MOV_Directory)
print(MOV_List)

['readnaversid1750oid448aid0000195268.mp4', 'readnaversid1750oid448aid0000195576.mp4', 'readnaversid1750oid448aid0000196603.mp4', 'video.mp4']


In [4]:
i = 0

for MOV in MOV_List:
    MOV_name = MOV_Directory + '/' + MOV
    video = cv2.VideoCapture(MOV_name) #video name change
    ret, frame = video.read()
    frame[0]
    cv2.imwrite("./First_scene/Picture%d.jpg" %i, frame) # [필수] 디렉토리 재설정
    video.release() 
    i += 1

## 1-2. 첫 장면에서부터 얼굴 2개가 검출될 경우 : 영상 편집에서 아예 제외 (제외된 케이스들은 STT로 처리)

In [5]:
First_img_Directory = './First_scene'
First_img_List = os.listdir(First_img_Directory)

print(First_img_List)

['Picture0.jpg', 'Picture1.jpg', 'Picture2.jpg', 'Picture3.jpg']


In [6]:
# 가중치 파일 경로
cascade_filename = 'haarcascade_frontalface_alt.xml'
# 모델 불러오기
cascade = cv2.CascadeClassifier(cascade_filename)

In [7]:
# 얼굴 검출 결과값 저장하는 리스트
results_list = []

# 처리하지 말아야 하는 동영상 index 저장하는 리스트
not_processing = []

# 얼굴 인식
for img in First_img_List:
    First_img_name = First_img_Directory + '/' + img
    First_img = cv2.imread(First_img_name)
    
    # 영상 압축
    img = cv2.resize(First_img,dsize=None,fx=1.0,fy=1.0)
    # 그레이 스케일 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    # cascade 얼굴 탐지 알고리즘 
    results = cascade.detectMultiScale(gray,            # 입력 이미지
                                       scaleFactor= 2,# 이미지 피라미드 스케일 factor
                                       minNeighbors=3,  # 인접 객체 최소 거리 픽셀
                                       minSize=(20,20)  # 탐지 객체 최소 크기
                                       )        
    
    results_list.append(results)

for i in range(len(results_list)):
    if len(results_list[i]) >= 2:
        not_processing.append(i)
        
# STT 사용해서 전처리해야 하는 영상들 index
print(not_processing)


[2, 3]


## 1-3. STT 없이 처리 가능한 동영상 리스트 추출

In [8]:
# STT 없이 처리 가능한 동영상 index 추출
need_idx = {}
for i in range(len(MOV_List)):
    need_idx[i] = MOV_List[i]

for i in not_processing:
    del need_idx[i]
        
print(need_idx)
need_idx = list(need_idx)

need_idx

{0: 'readnaversid1750oid448aid0000195268.mp4', 1: 'readnaversid1750oid448aid0000195576.mp4'}


[0, 1]

In [9]:
# STT 없어도 처리 가능한 동영상 이름 추출
need_processing = []
for i in need_idx:
    need_processing.append(MOV_List[i])
    
print(need_processing)

['readnaversid1750oid448aid0000195268.mp4', 'readnaversid1750oid448aid0000195576.mp4']


## 2. 앵커 1명의 얼굴만 나올 때 crop

In [10]:
# 가중치 파일 경로
# cascade_filename = 'haarcascade_frontalface_alt.xml'
# 모델 불러오기
# cascade = cv2.CascadeClassifier(cascade_filename)


for MOV in need_processing:
    MOV_name = MOV_Directory + '/' + MOV
    
    cap = cv2.VideoCapture(MOV_name)

    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')

    delay = round(1000/fps)

    i = 0
    out = cv2.VideoWriter('./News_Anchor/%s.avi' %MOV, fourcc, fps, (w, h))

    if not out.isOpened():
        print('File open failed!')
        cap.release()
        sys.exit()
    
    while True:
        # 캡처 이미지 불러오기
        ret, frame = cap.read()
    
        # 그레이 스케일 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        # cascade 얼굴 탐지 알고리즘 
        results = cascade.detectMultiScale(gray,            # 입력 이미지
                                            scaleFactor=2,# 이미지 피라미드 스케일 factor
                                            minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                            minSize=(30,30)) # 탐지 객체 최소 크기  
        
        for box in results:
            x, y, w, h = box
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            
        # cv2.imshow('facenet', frame)
        
        # 앵커 1명이 아닐 경우에 영상 자르기
        if len(results) != 1:
            out.write(frame)

        #  cap.release()
            out.release()
        #   cv2.destroyAllWindows()

    i += 1
            
                    
# [문제점 발생]
# 1명의 얼굴을 잘 인식하다가도 0명 또는 2명이라고 인식하는 사태가 발생하였음.
# -> 영상을 자르는 기준이 애매모호해짐... 
# -> 어떤 식으로 해야 앵커만 나오도록 영상을 잘 자를 수 있을까?
# -> 현재 상태로는 detectMultiScale 인수 조정을 고려하고 있음.

# 분명히 앵커 1명이 말하고 있는 상황임에도 불구하고...  
# 가끔 극초반에서부터 0명이라고 인식해서 
# 1초만에 영상이 잘리거나 아예 값이 없는 empty 오류가 발생하기도 함.

# 노트북으로 작업하면 컴퓨터 엄청 뜨거워짐. 
# -> 데스크탑으로 재작업 시도해야겠음...!

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
